# Preparando o ambiente

In [5]:
%pip install unidecode
import pandas as pd
import re
from unidecode import unidecode
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [6]:
caminho_csv = r"C:\dev\processo_seletivo_intuitivecare\datasets\rol_de_procedimentos_e_eventos_em_saude\rol_de_procedimentos_e_eventos_em_saude_com_colunas_renomeadas.csv"

df_procedimentos_e_eventos_em_saude = pd.read_csv(caminho_csv)

# Exploração do Dataset

### Melhorias a serem realizadas

1. Converter todos os textos para letras minúsculas.


2. Remoção de caracteres especiais


3. Excluir registros extras que foram inseridos durante a extração (a leitura dos nomes das colunas como registros adicionais).


4. Remoção de espaços em branco adicionais.


5. **Tratamento de dados nulos:**  
   Nas colunas `odontologica`, `ambulatorial`, `hospitalar_com_obstetricia`, `hospitalar_sem_obstetricia`, `plano_referencia` e `procedimento_de_alta_complexidade`:  
   - Substituir valores `NaN` por `False`.  
   - Alterar valores com iniciais presentes para `True`.

6. Alterar o formato da coluna "vigência" para o tipo `data`.

In [7]:
pd.option_context('display.max_rows', None, 'display.max_columns', None)

df_procedimentos_e_eventos_em_saude.head(20)

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,ACONSELHAMENTO GENÉTICO,NaN,NaN,NaN,AMB,HCO,HSO,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
1,ATENDIMENTO/ACOMPANHAMENTO EM HOSPITAL-DIA PSI...,NaN,NaN,NaN,NaN,HCO,HSO,REF,NaN,109,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
2,ATENDIMENTO INTEGRAL AO RECÉM-NASCIDO (SALA DE...,NaN,NaN,NaN,NaN,HCO,NaN,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
3,ATENDIMENTO MÉDICO DO INTENSIVISTA EM UTI GERA...,NaN,NaN,NaN,NaN,HCO,HSO,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
4,ATENDIMENTO PEDIÁTRICO A GESTANTES (3o TRIMESTRE),NaN,NaN,NaN,AMB,HCO,NaN,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
5,ATIVIDADE EDUCACIONAL PARA PLANEJAMENTO FAMILIAR,NaN,NaN,NaN,AMB,NaN,NaN,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
6,ATIVIDADE EDUCATIVA EM SAÚDE BUCAL,NaN,NaN,OD,NaN,NaN,NaN,NaN,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
7,ATIVIDADE EDUCATIVA EM ODONTOLOGIA PARA PAIS E...,NaN,NaN,OD,NaN,NaN,NaN,NaN,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
8,CONDICIONAMENTO EM ODONTOLOGIA (COM DIRETRIZ D...,NaN,NaN,OD,NaN,NaN,NaN,NaN,NaN,86,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
9,ESTABILIZAÇÃO DE PACIENTE POR MEIO DE CONTENÇÃ...,NaN,NaN,OD,NaN,NaN,NaN,NaN,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU\rACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS


In [8]:
df_procedimentos_e_eventos_em_saude.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3786 entries, 0 to 3785
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   procedimento                       3557 non-null   object
 1   rn\(alteracao)                     479 non-null    object
 2   vigencia                           472 non-null    object
 3   odontologica                       573 non-null    object
 4   ambulatorial                       1943 non-null   object
 5   hospitalar_com_obstetricia         3454 non-null   object
 6   hospitalar_sem_obstetricia         3389 non-null   object
 7   plano_referencia                   3327 non-null   object
 8   procedimento_de_alta_complexidade  586 non-null    object
 9   diretriz_de_utilizacao             370 non-null    object
 10  subgrupo                           3294 non-null   object
 11  grupo                              3293 non-null   object
 12  capitu

In [9]:
nulos_por_coluna = df_procedimentos_e_eventos_em_saude.isnull().sum()
print(nulos_por_coluna)


procedimento                          229
rn\(alteracao)                       3307
vigencia                             3314
odontologica                         3213
ambulatorial                         1843
hospitalar_com_obstetricia            332
hospitalar_sem_obstetricia            397
plano_referencia                      459
procedimento_de_alta_complexidade    3200
diretriz_de_utilizacao               3416
subgrupo                              492
grupo                                 493
capitulo                              496
dtype: int64


# Limpeza do dataset

### Converter textos para letras *minúscolas*

In [10]:
df_procedimentos_e_eventos_em_saude = df_procedimentos_e_eventos_em_saude.applymap(
    lambda x: x.lower() if isinstance(x, str) else x
)

C:\Users\danie\AppData\Local\Temp\ipykernel_788\656278208.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_procedimentos_e_eventos_em_saude = df_procedimentos_e_eventos_em_saude.applymap(


In [11]:
df_procedimentos_e_eventos_em_saude.head()

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,aconselhamento genético,NaN,NaN,NaN,amb,hco,hso,ref,NaN,NaN,"consultas, visitas hospitalares ou\racompanham...",procedimentos gerais,procedimentos gerais
1,atendimento/acompanhamento em hospital-dia psi...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,109,"consultas, visitas hospitalares ou\racompanham...",procedimentos gerais,procedimentos gerais
2,atendimento integral ao recém-nascido (sala de...,NaN,NaN,NaN,NaN,hco,NaN,ref,NaN,NaN,"consultas, visitas hospitalares ou\racompanham...",procedimentos gerais,procedimentos gerais
3,atendimento médico do intensivista em uti gera...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,NaN,"consultas, visitas hospitalares ou\racompanham...",procedimentos gerais,procedimentos gerais
4,atendimento pediátrico a gestantes (3o trimestre),NaN,NaN,NaN,amb,hco,NaN,ref,NaN,NaN,"consultas, visitas hospitalares ou\racompanham...",procedimentos gerais,procedimentos gerais


### Remoção de caracteres especiais e espaços em branco

In [12]:
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = unidecode(texto)
        texto = re.sub(r'[^a-zA-Z0-9\s]', ' ', texto)
        texto = ' '.join(texto.split())
    return texto

In [13]:
df_procedimentos_e_eventos_em_saude = df_procedimentos_e_eventos_em_saude.applymap(normalizar_texto)

C:\Users\danie\AppData\Local\Temp\ipykernel_788\3701091007.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_procedimentos_e_eventos_em_saude = df_procedimentos_e_eventos_em_saude.applymap(normalizar_texto)


In [14]:
df_procedimentos_e_eventos_em_saude.head()

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,aconselhamento genetico,NaN,NaN,NaN,amb,hco,hso,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
1,atendimento acompanhamento em hospital dia psi...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,109,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
2,atendimento integral ao recem nascido sala de ...,NaN,NaN,NaN,NaN,hco,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
3,atendimento medico do intensivista em uti gera...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
4,atendimento pediatrico a gestantes 3o trimestre,NaN,NaN,NaN,amb,hco,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais


### Remoçao de registros extras

In [15]:
nomes_colunas = df_procedimentos_e_eventos_em_saude.columns.tolist()

df_procedimentos_e_eventos_em_saude = df_procedimentos_e_eventos_em_saude[
    ~df_procedimentos_e_eventos_em_saude.apply(
        lambda row: any(cell in nomes_colunas for cell in row),
        axis=1
    )
]

In [16]:
df_procedimentos_e_eventos_em_saude = df_procedimentos_e_eventos_em_saude.reset_index(drop=True)
df_procedimentos_e_eventos_em_saude.head(20)

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,aconselhamento genetico,NaN,NaN,NaN,amb,hco,hso,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
1,atendimento acompanhamento em hospital dia psi...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,109,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
2,atendimento integral ao recem nascido sala de ...,NaN,NaN,NaN,NaN,hco,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
3,atendimento medico do intensivista em uti gera...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
4,atendimento pediatrico a gestantes 3o trimestre,NaN,NaN,NaN,amb,hco,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
5,atividade educacional para planejamento familiar,NaN,NaN,NaN,amb,NaN,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
6,atividade educativa em saude bucal,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
7,atividade educativa em odontologia para pais e...,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
8,condicionamento em odontologia com diretriz de...,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,86,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
9,estabilizacao de paciente por meio de contenca...,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais


In [17]:
df_procedimentos_e_eventos_em_saude.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3598 entries, 0 to 3597
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   procedimento                       3369 non-null   object
 1   rn\(alteracao)                     307 non-null    object
 2   vigencia                           284 non-null    object
 3   odontologica                       385 non-null    object
 4   ambulatorial                       1755 non-null   object
 5   hospitalar_com_obstetricia         3266 non-null   object
 6   hospitalar_sem_obstetricia         3201 non-null   object
 7   plano_referencia                   3139 non-null   object
 8   procedimento_de_alta_complexidade  398 non-null    object
 9   diretriz_de_utilizacao             182 non-null    object
 10  subgrupo                           3106 non-null   object
 11  grupo                              3105 non-null   object
 12  capitu

### Dados nulos

In [18]:
df_procedimentos_e_eventos_em_saude.head(20)

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,aconselhamento genetico,NaN,NaN,NaN,amb,hco,hso,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
1,atendimento acompanhamento em hospital dia psi...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,109,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
2,atendimento integral ao recem nascido sala de ...,NaN,NaN,NaN,NaN,hco,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
3,atendimento medico do intensivista em uti gera...,NaN,NaN,NaN,NaN,hco,hso,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
4,atendimento pediatrico a gestantes 3o trimestre,NaN,NaN,NaN,amb,hco,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
5,atividade educacional para planejamento familiar,NaN,NaN,NaN,amb,NaN,NaN,ref,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
6,atividade educativa em saude bucal,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
7,atividade educativa em odontologia para pais e...,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
8,condicionamento em odontologia com diretriz de...,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,86,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
9,estabilizacao de paciente por meio de contenca...,NaN,NaN,od,NaN,NaN,NaN,NaN,NaN,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais


In [19]:
colunas_para_converter = [
    'odontologica',
    'ambulatorial',
    'hospitalar_com_obstetricia',
    'hospitalar_sem_obstetricia',
    'plano_referencia',
    'procedimento_de_alta_complexidade'
]

for coluna in colunas_para_converter:
    df_procedimentos_e_eventos_em_saude[coluna] = df_procedimentos_e_eventos_em_saude[coluna].notna()

In [20]:
df_procedimentos_e_eventos_em_saude.head(20)

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,aconselhamento genetico,NaN,NaN,False,True,True,True,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
1,atendimento acompanhamento em hospital dia psi...,NaN,NaN,False,False,True,True,True,False,109,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
2,atendimento integral ao recem nascido sala de ...,NaN,NaN,False,False,True,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
3,atendimento medico do intensivista em uti gera...,NaN,NaN,False,False,True,True,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
4,atendimento pediatrico a gestantes 3o trimestre,NaN,NaN,False,True,True,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
5,atividade educacional para planejamento familiar,NaN,NaN,False,True,False,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
6,atividade educativa em saude bucal,NaN,NaN,True,False,False,False,False,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
7,atividade educativa em odontologia para pais e...,NaN,NaN,True,False,False,False,False,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
8,condicionamento em odontologia com diretriz de...,NaN,NaN,True,False,False,False,False,False,86,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
9,estabilizacao de paciente por meio de contenca...,NaN,NaN,True,False,False,False,False,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais


### Ajustar a coluna vigencia

In [21]:
df_procedimentos_e_eventos_em_saude['vigencia']= df_procedimentos_e_eventos_em_saude['vigencia'].str.replace(' ', '-', regex=False)

In [22]:
df_procedimentos_e_eventos_em_saude.head(20)

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,aconselhamento genetico,NaN,NaN,False,True,True,True,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
1,atendimento acompanhamento em hospital dia psi...,NaN,NaN,False,False,True,True,True,False,109,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
2,atendimento integral ao recem nascido sala de ...,NaN,NaN,False,False,True,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
3,atendimento medico do intensivista em uti gera...,NaN,NaN,False,False,True,True,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
4,atendimento pediatrico a gestantes 3o trimestre,NaN,NaN,False,True,True,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
5,atividade educacional para planejamento familiar,NaN,NaN,False,True,False,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
6,atividade educativa em saude bucal,NaN,NaN,True,False,False,False,False,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
7,atividade educativa em odontologia para pais e...,NaN,NaN,True,False,False,False,False,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
8,condicionamento em odontologia com diretriz de...,NaN,NaN,True,False,False,False,False,False,86,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
9,estabilizacao de paciente por meio de contenca...,NaN,NaN,True,False,False,False,False,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais


# Salvar csv

In [23]:
df_procedimentos_e_eventos_em_saude.to_csv('procedimentos_e_eventos_em_saude_limpo.csv', index=False)

In [24]:
df = pd.read_csv(r"C:\dev\processo_seletivo_intuitivecare\datasets\rol_de_procedimentos_e_eventos_em_saude\procedimentos_e_eventos_em_saude_limpo.csv")
df

,procedimento,rn\(alteracao),vigencia,odontologica,ambulatorial,hospitalar_com_obstetricia,hospitalar_sem_obstetricia,plano_referencia,procedimento_de_alta_complexidade,diretriz_de_utilizacao,subgrupo,grupo,capitulo
0,aconselhamento genetico,NaN,NaN,False,True,True,True,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
1,atendimento acompanhamento em hospital dia psi...,NaN,NaN,False,False,True,True,True,False,109,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
2,atendimento integral ao recem nascido sala de ...,NaN,NaN,False,False,True,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
3,atendimento medico do intensivista em uti gera...,NaN,NaN,False,False,True,True,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
4,atendimento pediatrico a gestantes 3o trimestre,NaN,NaN,False,True,True,False,True,False,NaN,consultas visitas hospitalares ou acompanhamen...,procedimentos gerais,procedimentos gerais
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,tonometria,NaN,NaN,False,True,True,True,True,False,NaN,procedimentos,exames especificos,procedimentos diagnosticos e terapeuticos
3594,tricograma para dermatologia mineralograma par...,NaN,NaN,False,True,True,True,True,False,NaN,procedimentos,exames especificos,procedimentos diagnosticos e terapeuticos
3595,urodinamica completa,NaN,NaN,False,True,True,True,True,False,NaN,procedimentos,exames especificos,procedimentos diagnosticos e terapeuticos
3596,urofluxometria,NaN,NaN,False,True,True,True,True,False,NaN,procedimentos,exames especificos,procedimentos diagnosticos e terapeuticos
